##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorBoard에 텍스트 데이터 표시하기

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/text_summaries"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tensorboard/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tensorboard/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tensorboard/text_summaries.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 개요

**TensorFlow Text Summary API**를 사용하면 임의의 텍스트를 쉽게 로깅하고 TensorBoard에서 볼 수 있습니다. 이는 입력 데이터를 샘플링 및 검사하거나 <a>실행 메타데이터</a> 또는 <a>생성된 텍스트를 기록</a>하는 데 대단히 유용할 수 있습니다. 또한, 진단 데이터를 텍스트로 로깅하여 모델 개발 과정에서 도움을 받을 수 있습니다.

이 자습서에서는 Text Summary API의 몇 가지의 간단한 사용 사례를 시도합니다.

## 설정

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
import tensorflow as tf

from datetime import datetime
import json
from packaging import version
import tempfile

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.5.0-dev20210219


## 짧은 텍스트 로깅하기

Text Summary API의 작동 방식을 이해하기 위해, 약간의 텍스트를 로깅하고 TensorBoard에 어떻게 표시되는지 확인하겠습니다.


In [4]:
my_text = "Hello world! 😃"

In [5]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/text_basics/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the text.
with file_writer.as_default():
  tf.summary.text("first_text", my_text, step=0)

이제 TensorBoard를 사용하여 텍스트를 검사합니다. UI가 모두 표시될 때까지 몇 초 정도 기다리세요.

In [6]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/text_simple.png?raw=1"/> -->

## 복수의 테스트 스트림 구성하기

복수의 테스트 스트림이 있는 경우 스칼라 혹은 기타 데이터와 같이 구성하는 데 도움이 되도록 별도의 네임스페이스에 보관할 수 있습니다.

다수의 단계로 로그 텍스트를 기록할 경우 TensorBoard는 프레젠테이션을 관리할 수 있도록 표시할 단계를 서브샘플링합니다. `--samples_per_plugin` 플래그를 사용하여 샘플링 속도를 제어할 수 있습니다.

In [7]:
# Sets up a second directory to not overwrite the first one.
logdir = "logs/multiple_texts/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the text.
with file_writer.as_default():
  with tf.name_scope("name_scope_1"):
    for step in range(20):
      tf.summary.text("a_stream_of_text", f"Hello from step {step}", step=step)
      tf.summary.text("another_stream_of_text", f"This can be kept separate {step}", step=step)
  with tf.name_scope("name_scope_2"):
    tf.summary.text("just_from_step_0", "This is an important announcement from step 0", step=0)
    

In [14]:
%tensorboard --logdir logs/multiple_texts --samples_per_plugin 'text=5'

## 마크다운 해석하기

TensorBoard는 텍스트 요약을 Markdown으로 번역합니다. 아래에 표시된 것처럼 풍부한 포맷을 사용하면 기록하는 데이터를 더 쉽게 읽고 이해할 수 있기 때문입니다. (마크다운 해석을 원하지 않는 경우 [이 이슈](https://github.com/tensorflow/tensorboard/issues/830) 에서 번역을 억제하는 방법을 참고하세요.)

In [9]:
# Sets up a third timestamped log directory under "logs"
logdir = "logs/markdown/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

some_obj_worth_noting = {
  "tfds_training_data": {
      "name": "mnist",
      "split": "train",
      "shuffle_files": "True",
  },
  "keras_optimizer": {
      "name": "Adagrad",
      "learning_rate": "0.001",
      "epsilon": 1e-07,
  },
  "hardware": "Cloud TPU",
}


# TODO: Update this example when TensorBoard is released with
# https://github.com/tensorflow/tensorboard/pull/4585
# which supports fenced codeblocks in Markdown.
def pretty_json(hp):
  json_hp = json.dumps(hp, indent=2)
  return "".join("\t" + line for line in json_hp.splitlines(True))

markdown_text = """
### Markdown Text

TensorBoard supports basic markdown syntax, including:

    preformatted code

**bold text**

| and | tables |
| ---- | ---------- |
| among | others |
"""

with file_writer.as_default():
  tf.summary.text("run_params", pretty_json(some_obj_worth_noting), step=0)
  tf.summary.text("markdown_jubiliee", markdown_text, step=0)
    

In [10]:
%tensorboard --logdir logs/markdown